In [48]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [49]:
# Define feature columns and load data

FEATURE_NAMES = [
    "hw_features",
    "req_permissions",
    "app_components",
    "intents",
    "api_calls",
    "used_permissions",
    "sus_api_calls",
    "urls",
]

SCALAR_COLS = []

# Load data
df = pd.read_csv("test.csv")
df = df.replace(np.nan, "")

display(df)

,sha256,hw_features,req_permissions,app_components,intents,api_calls,used_permissions,sus_api_calls,urls,malware,family
0,000068216bdb459df847bfdd67dd11069c3c50166db1ea...,"android.hardware.touchscreen,","android.permission.WRITE_EXTERNAL_STORAGE,andr...","paklena.batterydiviner.Settings,paklena.batter...","android.intent.category.HOME,android.intent.ac...","android/content/Context;->startService,android...","android.permission.VIBRATE,","Read/Write External Storage,getSystemService,",,0,Benign
1,0000764713b286cfe7e8e76c7038c92312977712d9c5a8...,"android.hardware.touchscreen,android.hardware....","android.permission.ACCESS_FINE_LOCATION,androi...","BootReceiver,ru.alpha.AlphaReceiver,ru.alpha.A...","android.intent.category.HOME,android.intent.ac...",android/net/ConnectivityManager;->getActiveNet...,"android.permission.INTERNET,android.permission...","getDeviceId,Read/Write External Storage,Obfusc...","http://m-001.net/i/,m-001.net,http://m-001.net...",1,Opfake
2,0000962c2c34de1ca0c329b18be7847459da2d9d14b6b2...,"android.hardware.touchscreen,android.hardware....","android.permission.INTERNET,",".index,index,","android.intent.action.MAIN,android.intent.cate...","android/webkit/WebView,","android.permission.INTERNET,","getSystemService,",,0,Benign
3,000167f1ff061ea91440c40659c11c2af160342fd2e493...,"android.hardware.touchscreen,android.hardware....","android.permission.INTERNET,android.permission...","NokyART,.NokyArtWidget$UpdateService,.NokyArt,...","android.intent.action.MAIN,android.intent.cate...",android/net/ConnectivityManager;->getNetworkIn...,"android.permission.INTERNET,android.permission...","getSystemService,",http://nokyart.n97.fr/Application/android/inde...,0,Benign
4,00017ed2c044caf7b1047184673ec3e11ab10ac0e64fb7...,"android.hardware.touchscreen,",,".TitlePageActivity,","android.intent.action.MAIN,android.intent.cate...",,,"printStackTrace,",,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...
994,01e78d647cabf702bdbbb3af46ab70f2cbb407dfe6c6f8...,"android.hardware.touchscreen,android.hardware....","android.permission.INTERNET,android.permission...",".EmotionSensor,","android.intent.action.MAIN,android.intent.cate...",android/net/ConnectivityManager;->getActiveNet...,"android.permission.ACCESS_NETWORK_STATE,androi...","getPackageInfo,getSystemService,Cipher(AES/CBC...",http://schemas.android.com/apk/lib/com.google....,0,Benign
995,01e7a30caff13ed959ec74a26ec27564dea94389200ed9...,"android.hardware.touchscreen,","android.permission.WRITE_EXTERNAL_STORAGE,",".main,","android.intent.action.MAIN,android.intent.cate...",android/content/ContentResolver;->openInputStr...,"android.permission.INTERNET,android.permission...","Read/Write External Storage,getSystemService,",,0,Benign
996,01e7a6ea485da0acbe7ac6f7c1dbb20df27faa3a06130f...,"android.hardware.touchscreen,android.hardware....","android.permission.INTERNET,android.permission...",".zaragoza_872,","android.intent.action.MAIN,android.intent.cate...",android/telephony/TelephonyManager;->getDevice...,"android.permission.ACCESS_FINE_LOCATION,androi...","getDeviceId,printStackTrace,Read/Write Externa...",,0,Benign
997,01e86ce55b5175a75ed513d452c5a7169a56d013d822f1...,"android.hardware.touchscreen,android.hardware....","android.permission.INTERNET,com.android.vendin...",".MainActivity,","android.intent.action.MAIN,android.intent.cate...","android/app/Activity;->startActivity,android/m...","android.permission.READ_CONTACTS,android.permi...","printStackTrace,getPackageInfo,getSystemServic...","http://market.android.com/details?id%3D,http:/...",0,Benign


Before preprocessing (raw df):

| idx | sha256 | list_of_strings_feature_1 | list_of_strings_feature_2 | ... | binary_target | target_type (string) |
|-----|--------|---------------------------|---------------------------|-----|---------------|----------------------|
|   0 | abcde  |            "s1,s2,s3,..." |            "s1,s2,s3,..." | ... |             0 |              "Type1" |
|   1 | cbedf1 |            "s1,s2,s3,..." |            "s1,s2,s3,..." | ... |             1 |              "Type2" | 
|   2 | fffff1 |                        "" |            "s1,s2,s3,..." | ... |             1 |              "Type1" | 

---

After preprocessing:

| idx | sha256 | list_of_strings_feature_1 | list_of_strings_feature_2 | ... | binary_target | target_type (string) |
|-----|--------|---------------------------|---------------------------|-----|---------------|----------------------|
|   0 | abcde0 |   ["s1", "s2", "s3", ...] |   ["s1", "s2", "s3", ...] | ... |             0 |              "Type1" |
|   1 | cbedf1 |   ["s1", "s2", "s3", ...] |   ["s1", "s2", "s3", ...] | ... |             1 |              "Type2" | 
|   2 | fffff1 |                        [] |   ["s1", "s2", "s3", ...] | ... |             1 |              "Type1" | 

In [50]:
# Preprocess data by converting comma-separated strings to lists (see diagram above ^)

df = df.assign(**{col: df[col].str.split(",") for col in FEATURE_NAMES})
df = df.assign(
    **{
        col: df[col].apply(lambda x: [f for f in x if f.strip() != ""])
        for col in FEATURE_NAMES
    }
)

display(df)

,sha256,hw_features,req_permissions,app_components,intents,api_calls,used_permissions,sus_api_calls,urls,malware,family
0,000068216bdb459df847bfdd67dd11069c3c50166db1ea...,[android.hardware.touchscreen],"[android.permission.WRITE_EXTERNAL_STORAGE, an...","[paklena.batterydiviner.Settings, paklena.batt...","[android.intent.category.HOME, android.intent....","[android/content/Context;->startService, andro...",[android.permission.VIBRATE],"[Read/Write External Storage, getSystemService]",[],0,Benign
1,0000764713b286cfe7e8e76c7038c92312977712d9c5a8...,"[android.hardware.touchscreen, android.hardwar...","[android.permission.ACCESS_FINE_LOCATION, andr...","[BootReceiver, ru.alpha.AlphaReceiver, ru.alph...","[android.intent.category.HOME, android.intent....",[android/net/ConnectivityManager;->getActiveNe...,"[android.permission.INTERNET, android.permissi...","[getDeviceId, Read/Write External Storage, Obf...","[http://m-001.net/i/, m-001.net, http://m-001....",1,Opfake
2,0000962c2c34de1ca0c329b18be7847459da2d9d14b6b2...,"[android.hardware.touchscreen, android.hardwar...",[android.permission.INTERNET],"[.index, index]","[android.intent.action.MAIN, android.intent.ca...",[android/webkit/WebView],[android.permission.INTERNET],[getSystemService],[],0,Benign
3,000167f1ff061ea91440c40659c11c2af160342fd2e493...,"[android.hardware.touchscreen, android.hardwar...","[android.permission.INTERNET, android.permissi...","[NokyART, .NokyArtWidget$UpdateService, .NokyA...","[android.intent.action.MAIN, android.intent.ca...",[android/net/ConnectivityManager;->getNetworkI...,"[android.permission.INTERNET, android.permissi...",[getSystemService],[http://nokyart.n97.fr/Application/android/ind...,0,Benign
4,00017ed2c044caf7b1047184673ec3e11ab10ac0e64fb7...,[android.hardware.touchscreen],[],[.TitlePageActivity],"[android.intent.action.MAIN, android.intent.ca...",[],[],[printStackTrace],[],0,Benign
...,...,...,...,...,...,...,...,...,...,...,...
994,01e78d647cabf702bdbbb3af46ab70f2cbb407dfe6c6f8...,"[android.hardware.touchscreen, android.hardwar...","[android.permission.INTERNET, android.permissi...",[.EmotionSensor],"[android.intent.action.MAIN, android.intent.ca...",[android/net/ConnectivityManager;->getActiveNe...,"[android.permission.ACCESS_NETWORK_STATE, andr...","[getPackageInfo, getSystemService, Cipher(AES/...",[http://schemas.android.com/apk/lib/com.google...,0,Benign
995,01e7a30caff13ed959ec74a26ec27564dea94389200ed9...,[android.hardware.touchscreen],[android.permission.WRITE_EXTERNAL_STORAGE],[.main],"[android.intent.action.MAIN, android.intent.ca...",[android/content/ContentResolver;->openInputSt...,"[android.permission.INTERNET, android.permissi...","[Read/Write External Storage, getSystemService]",[],0,Benign
996,01e7a6ea485da0acbe7ac6f7c1dbb20df27faa3a06130f...,"[android.hardware.touchscreen, android.hardwar...","[android.permission.INTERNET, android.permissi...",[.zaragoza_872],"[android.intent.action.MAIN, android.intent.ca...",[android/telephony/TelephonyManager;->getDevic...,"[android.permission.ACCESS_FINE_LOCATION, andr...","[getDeviceId, printStackTrace, Read/Write Exte...",[],0,Benign
997,01e86ce55b5175a75ed513d452c5a7169a56d013d822f1...,"[android.hardware.touchscreen, android.hardwar...","[android.permission.INTERNET, com.android.vend...",[.MainActivity],"[android.intent.action.MAIN, android.intent.ca...","[android/app/Activity;->startActivity, android...","[android.permission.READ_CONTACTS, android.per...","[printStackTrace, getPackageInfo, getSystemSer...","[http://market.android.com/details?id%3D, http...",0,Benign


In [51]:
from torchtext.vocab import build_vocab_from_iterator


def create_vocab_for_column(df, col, specials=["<PAD>", "<UNK>", "<EMPTY>"]):
    all_values = [val for seq in df[col] for val in seq]
    vocab = build_vocab_from_iterator([all_values], specials=specials)
    vocab.set_default_index(vocab["<UNK>"])
    return vocab


# Build vocabulary for each feature column
vocab_dict = {col: create_vocab_for_column(df, col) for col in FEATURE_NAMES}

print(vocab_dict["hw_features"].get_itos())

['<PAD>', '<UNK>', '<EMPTY>', 'android.hardware.touchscreen', 'android.hardware.screen.portrait', 'android.hardware.location', 'android.hardware.location.network', 'android.hardware.location.gps', 'android.hardware.telephony', 'android.hardware.wifi', 'android.hardware.screen.landscape', 'android.hardware.camera', 'android.hardware.camera.autofocus', 'android.hardware.microphone', 'android.hardware.bluetooth', 'android.software.live_wallpaper', 'android.hardware.touchscreen.multitouch', 'android.hardware.sensor.accelerometer', 'ACCESS_COARSE_LOCATION', 'READ_SMS', 'RECEIVE_SMS', 'android.hardware.camera.flash', 'android.hardware.nfc', 'android.hardware.sensor.compass']


In [52]:
from torch.utils.data import Dataset


# Dataset wraper for the Debrim dataset
class DebrimDataset(Dataset):
    def __init__(self, df, vocab_dict, scalar_cols, label_col):
        self.df = df
        self.vocab_dict = vocab_dict
        self.scalar_cols = scalar_cols
        self.label_col = label_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        features = {
            col: torch.tensor(self.vocab_dict[col](row[col]), dtype=torch.long)
            for col in self.vocab_dict.keys()
        }

        scalars = {
            col: torch.tensor(row[col], dtype=torch.float) for col in self.scalar_cols
        }

        label = torch.tensor(row[self.label_col], dtype=torch.long)

        return features, scalars, label


# Function to handle batching with variable sequence lengths
def collate_fn(batch):
    features, scalars, label = zip(*batch)

    # Pad sequences to the same length
    features = {
        col: torch.nn.utils.rnn.pad_sequence(
            [f[col] for f in features], batch_first=True
        )
        for col in features[0].keys()
    }

    # Stack scalars and label
    scalars = {col: torch.stack([s[col] for s in scalars]) for col in scalars[0].keys()}

    label = torch.stack(label)

    return features, scalars, label

In [53]:
import torch.nn as nn


# Model definition
class DebrimModel(nn.Module):
    def __init__(
        self,
        vocab_dict,
        scalar_dim,
        embedding_dim=128,
        seq_pooling="mean",
        hidden_dim=[128, 64],
        n_classes=2,
        dropout=0.5,
    ):
        super().__init__()

        self.seq_pooling = seq_pooling

        # Embedding layer for each feature
        self.embedders = nn.ModuleDict(
            {
                col: nn.Embedding(
                    num_embeddings=len(vocab),
                    embedding_dim=embedding_dim,
                    padding_idx=vocab["<PAD>"],
                )
                for col, vocab in vocab_dict.items()
            }
        )

        total_dim = embedding_dim * len(vocab_dict) + scalar_dim

        # MLP layers
        layers = []
        dims = [total_dim] + hidden_dim
        for in_d, out_d in zip(dims, dims[1:]):
            layers += [nn.Linear(in_d, out_d), nn.ReLU(), nn.Dropout(dropout)]

        layers.append(nn.Linear(dims[-1], n_classes))
        self.mlp = nn.Sequential(*layers)

    def forward(self, seq_feats, scalars):
        pooled = []
        for col, seq in seq_feats.items():
            emb = self.embedders[col](seq)              # [Batch, Length, Embed_dim]

            if self.seq_pooling == "mean":
                # Mean pooling
                mask = (seq != 0).unsqueeze(-1)         # [B, L, 1]
                summed = (emb * mask).sum(dim=1)        # [B, E]
                counts = mask.sum(dim=1).clamp(min=1)
                pooled.append(summed / counts)               # [B, E]
            else:
                # Max pooling
                emb = emb.masked_fill(seq.unsqueeze(-1) == 0, float("-inf"))
                pooled.append(emb.max(dim=1).values)         # [B, E]

        scalar_vecs = [scalars[col] for col in scalars]
        if scalar_vecs:
            scalars = torch.cat(scalar_vecs, dim=1)
            pooled.append(scalars)

        x = torch.cat(pooled, dim=1)                 # [B, total_dim]
        logits = self.mlp(x)                            # [B, n_classes]
        return logits

In [54]:
# Basic training loop

from torch.utils.data import DataLoader
import torch.optim as optim

# Hyperparameters
BATCH_SIZE = 64
LR = 1e-3
EPOCHS = 20

# Create dataset & loader
dataset = DebrimDataset(df, vocab_dict, scalar_cols=SCALAR_COLS, label_col="malware")
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

# Instantiate model
model = DebrimModel(vocab_dict, scalar_dim=len(SCALAR_COLS))
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

# Check for CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

# Training loop
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for seq_feats, scalars, label in loader:
        # Move data to device
        seq_feats = {k: v.to(device) for k, v in seq_feats.items()}
        scalars = {k: v.to(device) for k, v in scalars.items()}
        label = label.to(device)

        # Forward pass
        optimizer.zero_grad()
        logits = model(seq_feats, scalars)
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f"Epoch {epoch + 1}/{EPOCHS} — Loss: {total_loss / len(loader):.4f}")

Using device: cuda
Epoch 1/20 — Loss: 0.3125
Epoch 2/20 — Loss: 0.1653
Epoch 3/20 — Loss: 0.1082
Epoch 4/20 — Loss: 0.0950
Epoch 5/20 — Loss: 0.0777
Epoch 6/20 — Loss: 0.0635
Epoch 7/20 — Loss: 0.0581
Epoch 8/20 — Loss: 0.0467
Epoch 9/20 — Loss: 0.0451
Epoch 10/20 — Loss: 0.0301
Epoch 11/20 — Loss: 0.0241
Epoch 12/20 — Loss: 0.0171
Epoch 13/20 — Loss: 0.0107
Epoch 14/20 — Loss: 0.0150
Epoch 15/20 — Loss: 0.0123
Epoch 16/20 — Loss: 0.0057
Epoch 17/20 — Loss: 0.0100
Epoch 18/20 — Loss: 0.0073
Epoch 19/20 — Loss: 0.0036
Epoch 20/20 — Loss: 0.0019


In [55]:
# K-fold cross-validation with early stopping

from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, f1_score

from torch.utils.data import DataLoader
import torch.nn.functional as F

# Hyperparameters
BATCH_SIZE = 64
LR = 1e-3
EPOCHS = 20
FOLDS = 5
PATIENCE = 5

# Check for CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

# Compute class weights to handle imbalanced data
y = df["malware"].values 
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

fold_results = []

# Training and evaluation for each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
    print(f"\n=== Fold {fold + 1}/{FOLDS} ===")

    # Split data into train and validation sets and create datasets / loaders
    train_df = df.iloc[train_idx].reset_index(drop=True)
    val_df = df.iloc[val_idx].reset_index(drop=True)

    train_dataset = DebrimDataset(
        train_df, vocab_dict, scalar_cols=SCALAR_COLS, label_col="malware"
    )
    val_dataset = DebrimDataset(
        val_df, vocab_dict, scalar_cols=SCALAR_COLS, label_col="malware"
    )

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
    )
    val_loader = DataLoader(
        val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
    )

    # Instantiate model
    model = DebrimModel(vocab_dict, scalar_dim=len(SCALAR_COLS)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = torch.nn.CrossEntropyLoss(weight=weights_tensor)

    best_val_loss = float("inf")
    epochs_no_improve = 0

    # Training loop
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0

        # Same as before, forward pass
        for seq_feats, scalars, label in train_loader:
            seq_feats = {k: v.to(device) for k, v in seq_feats.items()}
            scalars = {k: v.to(device) for k, v in scalars.items()}
            label = label.to(device)

            optimizer.zero_grad()
            logits = model(seq_feats, scalars)
            loss = criterion(logits, label)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0
        y_true, y_pred, y_prob = [], [], []

        with torch.no_grad():
            for seq_feats, scalars, label in val_loader:
                seq_feats = {k: v.to(device) for k, v in seq_feats.items()}
                scalars = {k: v.to(device) for k, v in scalars.items()}
                label = label.to(device)

                logits = model(seq_feats, scalars)
                loss = criterion(logits, label)
                val_loss += loss.item()

                preds = torch.argmax(logits, dim=1)
                probs = F.softmax(logits, dim=1)[:, 1]  # Probability of positive class

                y_true.extend(label.cpu().tolist())
                y_pred.extend(preds.cpu().tolist())
                y_prob.extend(probs.cpu().tolist())

        # Calculate metrics
        avg_val_loss = val_loss / len(val_loader)
        
        # Weighted F1 score is a lot better than binary F1 score due to class imbalance
        f1 = f1_score(y_true, y_pred, average="weighted") 
        report = classification_report(y_true, y_pred, output_dict=True)

        print(
            f"Epoch {epoch + 1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}, F1 = {f1:.4f}"
        )

        # Check for improvement
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_metrics = (f1, report)
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Early stopping
        if epochs_no_improve >= PATIENCE:
            print("Early stopping.")
            break

    fold_results.append(
        {"fold": fold + 1, "f1": best_metrics[0], "report": best_metrics[1]}
    )

Using device: cuda

=== Fold 1/5 ===
Epoch 1: Train Loss = 0.6003, Val Loss = 0.4915, F1 = 0.9326
Epoch 2: Train Loss = 0.4240, Val Loss = 0.3098, F1 = 0.9471
Epoch 3: Train Loss = 0.2790, Val Loss = 0.2233, F1 = 0.9510
Epoch 4: Train Loss = 0.2081, Val Loss = 0.1969, F1 = 0.9632
Epoch 5: Train Loss = 0.1961, Val Loss = 0.1577, F1 = 0.9432
Epoch 6: Train Loss = 0.1482, Val Loss = 0.2085, F1 = 0.9674
Epoch 7: Train Loss = 0.1480, Val Loss = 0.1676, F1 = 0.9318
Epoch 8: Train Loss = 0.1079, Val Loss = 0.2477, F1 = 0.9709
Epoch 9: Train Loss = 0.1065, Val Loss = 0.1734, F1 = 0.9550
Epoch 10: Train Loss = 0.0744, Val Loss = 0.1983, F1 = 0.9632
Early stopping.

=== Fold 2/5 ===
Epoch 1: Train Loss = 0.6301, Val Loss = 0.5787, F1 = 0.9273
Epoch 2: Train Loss = 0.3910, Val Loss = 0.3827, F1 = 0.9531
Epoch 3: Train Loss = 0.2501, Val Loss = 0.3521, F1 = 0.9571
Epoch 4: Train Loss = 0.1859, Val Loss = 0.3129, F1 = 0.9503
Epoch 5: Train Loss = 0.1253, Val Loss = 0.4090, F1 = 0.9710
Epoch 6: Trai

In [56]:
# Summary of results
print("\n=== Cross-validation results ===")
for result in fold_results:
    print(f"Fold {result['fold']}: F1 Score = {result['f1']:.4f}")
    print("Classification Report:")
    print(pd.DataFrame(result["report"]).T, end="\n\n")
    print("===" * 10)

print("\n=== Overall Results ===")
avg_f1 = np.mean([result["f1"] for result in fold_results])
print(f"Average F1 Score = {avg_f1:.4f}")


=== Cross-validation results ===
Fold 1: F1 Score = 0.9432
Classification Report:
              precision    recall  f1-score  support
0              0.994286  0.935484  0.963989  186.000
1              0.520000  0.928571  0.666667   14.000
accuracy       0.935000  0.935000  0.935000    0.935
macro avg      0.757143  0.932028  0.815328  200.000
weighted avg   0.961086  0.935000  0.943176  200.000

Fold 2: F1 Score = 0.9503
Classification Report:
              precision    recall  f1-score  support
0              0.988889  0.951872  0.970027  187.000
1              0.550000  0.846154  0.666667   13.000
accuracy       0.945000  0.945000  0.945000    0.945
macro avg      0.769444  0.899013  0.818347  200.000
weighted avg   0.960361  0.945000  0.950309  200.000

Fold 3: F1 Score = 0.9439
Classification Report:
              precision    recall  f1-score  support
0              0.994444  0.932292  0.962366   192.00
1              0.350000  0.875000  0.500000     8.00
accuracy       0.93000

In [57]:
# Prediction function for inference
def predict(model, df, vocab_dict, scalar_cols, device, batch_size=64):
    model.eval()
    dataset = DebrimDataset(
        df, vocab_dict, scalar_cols=scalar_cols, label_col="malware"
    )
    loader = DataLoader(
        dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
    )

    all_preds = []
    all_probs = []

    with torch.no_grad():
        for seq_feats, scalars, _ in loader:
            seq_feats = {k: v.to(device) for k, v in seq_feats.items()}
            scalars = {k: v.to(device) for k, v in scalars.items()}

            logits = model(seq_feats, scalars)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_preds.extend(preds.cpu().tolist())
            all_probs.extend(probs.cpu().tolist())

    return all_preds, all_probs

In [58]:
# Example prediction on new data

# TODO: Example usage
new_df = pd.read_csv("new_test.csv")
new_df = new_df.replace(np.nan, "")
new_df = new_df.assign(**{col: new_df[col].str.split(",") for col in FEATURE_NAMES})
new_df = new_df.assign(
    **{
        col: new_df[col].apply(lambda x: [f for f in x if f.strip() != ""])
        for col in FEATURE_NAMES
    }
)

# Get predictions using trained model
preds, probs = predict(model, new_df, vocab_dict, SCALAR_COLS, device)

# Results
for i, (p, pr) in enumerate(zip(preds, probs)):
    print(f"Sample {i} — Predicted: {p}, Probabilities: {pr}")

Sample 0 — Predicted: 1, Probabilities: [0.00011553068907232955, 0.9998844861984253]


In [59]:
# Save / Load model
import os


os.makedirs("./nn_models", exist_ok=True)

# Save model weights to disk
torch.save(model.state_dict(), "./nn_models/debrim_model.pt")

# Load model weights from disk
model = DebrimModel(vocab_dict, scalar_dim=len(SCALAR_COLS))
model.load_state_dict(torch.load("./nn_models/debrim_model.pt"))
model.to(device)
model.eval()

DebrimModel(
  (embedders): ModuleDict(
    (hw_features): Embedding(24, 128, padding_idx=0)
    (req_permissions): Embedding(183, 128, padding_idx=0)
    (app_components): Embedding(3968, 128, padding_idx=0)
    (intents): Embedding(206, 128, padding_idx=0)
    (api_calls): Embedding(127, 128, padding_idx=0)
    (used_permissions): Embedding(43, 128, padding_idx=0)
    (sus_api_calls): Embedding(117, 128, padding_idx=0)
    (urls): Embedding(6174, 128, padding_idx=0)
  )
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=64, out_features=2, bias=True)
  )
)

# CLASIC ML MODELS

In [60]:
# Clasical ML models

# Extract embeddings from the neural network model
def extract_embeddings(model, df, vocab_dict, scalar_cols, device, batch_size=64):
    model.eval()
    dataset = DebrimDataset(
        df, vocab_dict, scalar_cols=scalar_cols, label_col="malware"
    )
    loader = DataLoader(
        dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
    )
    
    all_embeddings = []
    all_labels = []
    
    with torch.no_grad():
        for seq_feats, scalars, labels in loader:
            seq_feats = {k: v.to(device) for k, v in seq_feats.items()}
            scalars = {k: v.to(device) for k, v in scalars.items()}
            
            # Get embeddings for each feature
            pooled_embeddings = []
            for col, seq in seq_feats.items():
                emb = model.embedders[col](seq)
                
                # Apply the same pooling as in the model
                if model.seq_pooling == "mean":
                    mask = (seq != 0).unsqueeze(-1)
                    summed = (emb * mask).sum(dim=1)
                    counts = mask.sum(dim=1).clamp(min=1)
                    pooled_embeddings.append((summed / counts).cpu().tolist())
                else:
                    emb = emb.masked_fill(seq.unsqueeze(-1) == 0, float("-inf"))
                    pooled_embeddings.append(emb.max(dim=1).values.cpu().tolist())
            
            # Concatenate all embeddings
            batch_embeddings = np.concatenate(pooled_embeddings, axis=1)
            all_embeddings.append(batch_embeddings)
            all_labels.append(labels.cpu().tolist())
    
    return np.vstack(all_embeddings), np.concatenate(all_labels)

In [61]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


# Train classical models
def train_classical_models(models, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        report = classification_report(y_test, model.predict(X_test), output_dict=True)
        results[name] = report
    
    return results

# Train classical models using cross-validation
def train_classical_models_cv(models, X, y, cv=5):
    results = {}
    for name, model in models.items():
        # Fit the model using cross-validation and compute the classification report
        scores = cross_val_score(model, X, y, cv=cv, scoring="f1_weighted")
        report = classification_report(y, model.predict(X), output_dict=True)
        results[name] = report
    
    return results

In [62]:
# Example usage of the loaded model for prediction

X, y = extract_embeddings(model, df, vocab_dict, SCALAR_COLS, device)
print(f"Extracted embeddings shape: {X.shape}, Labels shape: {y.shape}")

# Define classical models
models = {
    "random_forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "decision_tree": DecisionTreeClassifier(random_state=42),
    "knn": KNeighborsClassifier(n_neighbors=5)
}

# Train classical models on the extracted embeddings
classic_results = train_classical_models(models, X, y)

# Train classical models using cross-validation
cv_results = train_classical_models_cv(models, X, y, cv=5)

Extracted embeddings shape: (999, 1024), Labels shape: (999,)


In [63]:
# Show results
print("\n=== Classical Models Results ===")
for name, report in classic_results.items():
    print(f"Model: '{name}' - Report:")
    print(pd.DataFrame(report).T, end="\n\n")
    print("===" * 10)
    
# Show cross-validation results
print("\n=== Classical Models Cross-Validation Results ===")
for name, report in cv_results.items():
    print(f"Model: '{name}' - Report:")
    print(pd.DataFrame(report).T, end="\n\n")
    print("===" * 10)


=== Classical Models Results ===
Model: 'random_forest' - Report:
              precision    recall  f1-score  support
0              0.948718  0.994624  0.971129  186.000
1              0.800000  0.285714  0.421053   14.000
accuracy       0.945000  0.945000  0.945000    0.945
macro avg      0.874359  0.640169  0.696091  200.000
weighted avg   0.938308  0.945000  0.932623  200.000

Model: 'decision_tree' - Report:
              precision    recall  f1-score  support
0              0.962162  0.956989  0.959569  186.000
1              0.466667  0.500000  0.482759   14.000
accuracy       0.925000  0.925000  0.925000    0.925
macro avg      0.714414  0.728495  0.721164  200.000
weighted avg   0.927477  0.925000  0.926192  200.000

Model: 'knn' - Report:
              precision    recall  f1-score  support
0              0.953608  0.994624  0.973684   186.00
1              0.833333  0.357143  0.500000    14.00
accuracy       0.950000  0.950000  0.950000     0.95
macro avg      0.893471  0.

In [64]:
import joblib
import os


os.makedirs("./classic_models", exist_ok=True)

# Save the classical models
for name, model in models.items():
    joblib.dump(model, f"./classic_models/{name}_model.pkl")
    
# Load the classical models
for name in models.keys():
    models[name] = joblib.load(f"./classic_models/{name}_model.pkl")